# Exploration API

In [1]:
import requests
from pprint import pprint
import time
import pandas as pd
import plotly.express as px
import streamlit as st
import plotly.graph_objects as go
import numpy as np

In [2]:
user = "Gotwiller"

In [3]:
df = pd.read_csv(f"../Data/{user}.csv")
max_timestamp = df['uts'].max()
max_timestamp

1764571939

## Liste des amis suivis

In [4]:
from pprint import pprint

api_key = "ea311d73665c24b237160f90bcb986ff"
artist = "Daft Punk"
method = "artist.getInfo"

url = f"http://ws.audioscrobbler.com/2.0/?method=user.getfriends&user={user}&api_key={api_key}&format=json"
# Envoi d'une requête GET
response = requests.get(url)

# Vérification du statut de la réponse
if response.status_code == 200:
    # Affichage des données JSON
    data = response.json()
    for user in data["friends"]["user"]:
        print(user["name"])
else:
    print(f"Erreur : {response.status_code}")

Mehdinho16
anessrb
LightAkabane
Kallikall
Boborlegamorf
amoham16
Cassy1508
Ayamathers
drikoooooo
tzisteph
Kronyx67
jardinier38
clara_llz
Petah_Parkour
Grimkujow
redneck2_0
GunLock38
Alxstg
Charloops
Iandir
IcaRio_
Shironekii
Watch77


In [11]:
api_key = "ea311d73665c24b237160f90bcb986ff"
method = "user.getInfo"

url = f"http://ws.audioscrobbler.com/2.0/?method={method}&user={user}&api_key={api_key}&format=json"
# Envoi d'une requête GET
response = requests.get(url)

# Vérification du statut de la réponse
if response.status_code == 200:
    # Affichage des données JSON
    data = response.json()
    playcount = int(data["user"]["playcount"])
    pprint(playcount)
else:
    print(f"Erreur : {response.status_code}")

Erreur : 400


In [14]:
method = "user.getrecenttracks"
limit = 200 
timestamp_start = 1756731733
playcount = 105000

all_tracks = []
page = 1
total_pages = playcount / 200 + 1

while page <= total_pages:
    url = f"http://ws.audioscrobbler.com/2.0/?method={method}&limit={limit}&user={user}&page={page}&from={timestamp_start}&api_key={api_key}&format=json"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        tracks = data['recenttracks']['track']
        all_tracks.extend(tracks)
        
        # mise à jour du nombre total de pages
        total_pages = int(data['recenttracks']['@attr']['totalPages'])
        print(f"Page {page}/{total_pages} récupérée, total tracks: {len(all_tracks)}")
        
        page += 1
        time.sleep(0.2)  # pour ne pas spammer l'API
    else:
        print(f"Erreur à la page {page} : {response.status_code}")
        break

# Affichage final de tous les tracks récupérés
pprint(all_tracks)


Erreur à la page 1 : 400
[]


## Changement de format

In [13]:
import csv

# Supposons que tes données JSON soient dans une liste appelée all_tracks
# all_tracks = [...]  # tes données JSON récupérées

output_file = "lastfm_tracks_filtered.csv"

with open(output_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # Écriture de l'en-tête
    writer.writerow(["uts", "utc_time", "artist", "artist_mbid", "album", "album_mbid", "track", "track_mbid"])
    
    for track in all_tracks:
        # Exclure les tracks en cours de lecture
        nowplaying = track.get('@attr', {}).get('nowplaying', 'false')
        if nowplaying.lower() == 'true':
            continue  # passer au track suivant
        
        # Certains tracks peuvent ne pas avoir de date
        date_info = track.get("date", {})
        uts = date_info.get("uts", "")
        utc_time = date_info.get("#text", "")
        
        artist_name = track["artist"].get("#text", "")
        artist_mbid = track["artist"].get("mbid", "")
        album_name = track["album"].get("#text", "")
        album_mbid = track["album"].get("mbid", "")
        track_name = track.get("name", "")
        track_mbid = track.get("mbid", "")
        
        writer.writerow([uts, utc_time, artist_name, artist_mbid, album_name, album_mbid, track_name, track_mbid])

print(f"CSV généré sans les morceaux en lecture : {output_file}")


CSV généré sans les morceaux en lecture : lastfm_tracks_filtered.csv


## Concatenation csv

In [ ]:
new_df = pd.read_csv("lastfm_tracks_filtered.csv")
old_df = pd.read_csv(f"../Data/{user}.csv")
combined_df = pd.concat([old_df, new_df]).drop_duplicates().reset_index(drop=True)
combined_df.to_csv(f"../Data/{user}.csv", index=False)

# Exploration Visus Solo

### Téléchargement des données 

In [16]:
df = pd.read_csv("../data/Gotwiller.csv")

# 2. Aperçu rapide
df.head()        # premières lignes

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1764571939,"01 Dec 2025, 06:52",Madonna,79239441-bfd5-4981-a70c-55c3f15c1287,Like a Virgin,055e17bc-fe18-4981-9da6-773972d167a3,Material Girl,9607fd1a-19c6-45b8-9445-cb351d4405c2
1,1764571708,"01 Dec 2025, 06:48",Fleetwood Mac,NaN,Tango in the Night (Deluxe Edition),NaN,Everywhere - 2017 Remaster,NaN
2,1764571509,"01 Dec 2025, 06:45",Kate Bush,NaN,Never for Ever (2018 Remaster),NaN,Babooshka - 2018 Remaster,NaN
3,1764571264,"01 Dec 2025, 06:41",M83,6d7b7cd4-254b-4c25-83f6-dd20f98ceacd,"Hurry Up, We're Dreaming",0b87ebcf-216b-4255-9c19-93c12861f173,Midnight City,9a952210-caf6-407d-8748-b89fcaca3bb7
4,1764571056,"01 Dec 2025, 06:37",The Cure,69ee3720-a7cb-4402-b48d-a02c366f2bcf,Disintegration (Deluxe Edition),NaN,Lovesong,NaN


### Ajout de Colonne supplémentaires

In [17]:
# Convertir la colonne de temps
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# Ajouter des colonnes utiles
df["date"] = df["utc_time"].dt.date
df["year"] = df["utc_time"].dt.year
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()
df["week"] = df["utc_time"].dt.isocalendar().week  # Numéro de semaine ISO

In [18]:
df.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,year,hour,weekday,week
0,1764571939,2025-12-01 06:52:00,Madonna,79239441-bfd5-4981-a70c-55c3f15c1287,Like a Virgin,055e17bc-fe18-4981-9da6-773972d167a3,Material Girl,9607fd1a-19c6-45b8-9445-cb351d4405c2,2025-12-01,2025,6,Monday,49
1,1764571708,2025-12-01 06:48:00,Fleetwood Mac,NaN,Tango in the Night (Deluxe Edition),NaN,Everywhere - 2017 Remaster,NaN,2025-12-01,2025,6,Monday,49
2,1764571509,2025-12-01 06:45:00,Kate Bush,NaN,Never for Ever (2018 Remaster),NaN,Babooshka - 2018 Remaster,NaN,2025-12-01,2025,6,Monday,49
3,1764571264,2025-12-01 06:41:00,M83,6d7b7cd4-254b-4c25-83f6-dd20f98ceacd,"Hurry Up, We're Dreaming",0b87ebcf-216b-4255-9c19-93c12861f173,Midnight City,9a952210-caf6-407d-8748-b89fcaca3bb7,2025-12-01,2025,6,Monday,49
4,1764571056,2025-12-01 06:37:00,The Cure,69ee3720-a7cb-4402-b48d-a02c366f2bcf,Disintegration (Deluxe Edition),NaN,Lovesong,NaN,2025-12-01,2025,6,Monday,49


In [19]:
# --- Option : sélectionner une année spécifique ---
year_selected = 2025
df_year = df[df["year"] == year_selected]

jours_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# --- Regrouper par semaine et jour ---
heatmap_data = df_year.groupby(['week', 'weekday']).size().reset_index(name='plays')

# Pivot pour créer la matrice
matrix = heatmap_data.pivot(index='weekday', columns='week', values='plays').reindex(jours_order)

# Remplacer les 0 par NaN pour laisser les cases vides
matrix = matrix.replace(0, np.nan)

# --- Étiquettes pour axes ---
semaines = [f"W{w}" for w in matrix.columns]
jours = matrix.index.tolist()

# --- Création de la heatmap ---
fig = px.imshow(
    matrix.values,
    x=semaines,
    y=jours,
    text_auto=True,
    color_continuous_scale='Turbo'
)

# --- Mettre le fond blanc pour les NaN ---
fig.update_traces(
    hovertemplate="%{y}, %{x}: %{z}<extra></extra>",
    zmin=0
)
fig.update_layout(
    title=f"Weekly activity per week - Year {year_selected}",
    xaxis_title="Week of the year",
    yaxis_title="Day of the week",
    plot_bgcolor='white',   # fond de la zone de tracé
    paper_bgcolor='white'   # fond autour du graphique
)

fig.show()

In [21]:

artist_counts = df["artist"].value_counts().reset_index()
artist_counts.columns = ["artist", "plays"]
fig = px.bar(
    artist_counts.head(10),
    x="plays",
    y="artist",
    orientation="h",
    title="Top 10 artistes écoutés",
    color="plays",
    color_continuous_scale="viridis"
)
fig.show()


In [22]:
hourly = df.groupby("hour").size().reset_index(name="plays")
px.area(hourly, x="hour", y="plays", title="Répartition des écoutes par heure", markers=True)


# Exploration Visus Duo

In [24]:
df_kronyx = pd.read_csv("../data/Kronyx67.csv")
df_gotwiller = pd.read_csv("../data/Gotwiller.csv")

# 2. Aperçu rapide
df_kronyx.head()      

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1764516892,"30 Nov 2025, 15:34",Nava,3654dbc1-1f71-4686-8a9d-c8d63f416598,Tic Tac,NaN,Tic Tac,NaN
1,1764516699,"30 Nov 2025, 15:31",Yuston XIII,NaN,Que des cendres,NaN,Que des cendres,NaN
2,1764516456,"30 Nov 2025, 15:27",Mathilde,f7b37e03-d216-4144-901a-cebca6426416,La nuit · Le jour,NaN,Libre !,NaN
3,1764516286,"30 Nov 2025, 15:24",Théadule,NaN,Un Monde Parfait,NaN,Un Monde Parfait,NaN
4,1764516117,"30 Nov 2025, 15:21",Louisette & Amaury,NaN,Mieux,NaN,Mieux,NaN


In [25]:
df_gotwiller.head()  

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1764571939,"01 Dec 2025, 06:52",Madonna,79239441-bfd5-4981-a70c-55c3f15c1287,Like a Virgin,055e17bc-fe18-4981-9da6-773972d167a3,Material Girl,9607fd1a-19c6-45b8-9445-cb351d4405c2
1,1764571708,"01 Dec 2025, 06:48",Fleetwood Mac,NaN,Tango in the Night (Deluxe Edition),NaN,Everywhere - 2017 Remaster,NaN
2,1764571509,"01 Dec 2025, 06:45",Kate Bush,NaN,Never for Ever (2018 Remaster),NaN,Babooshka - 2018 Remaster,NaN
3,1764571264,"01 Dec 2025, 06:41",M83,6d7b7cd4-254b-4c25-83f6-dd20f98ceacd,"Hurry Up, We're Dreaming",0b87ebcf-216b-4255-9c19-93c12861f173,Midnight City,9a952210-caf6-407d-8748-b89fcaca3bb7
4,1764571056,"01 Dec 2025, 06:37",The Cure,69ee3720-a7cb-4402-b48d-a02c366f2bcf,Disintegration (Deluxe Edition),NaN,Lovesong,NaN


In [26]:
for df in [df_kronyx, df_gotwiller]:
    df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")
    # Ajouter des colonnes utiles
    df["date"] = df["utc_time"].dt.date
    df["hour"] = df["utc_time"].dt.hour
    df["weekday"] = df["utc_time"].dt.day_name()
    df["minute"] = df["utc_time"].dt.minute

In [27]:
df_kronyx.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,hour,weekday,minute
0,1764516892,2025-11-30 15:34:00,Nava,3654dbc1-1f71-4686-8a9d-c8d63f416598,Tic Tac,NaN,Tic Tac,NaN,2025-11-30,15,Sunday,34
1,1764516699,2025-11-30 15:31:00,Yuston XIII,NaN,Que des cendres,NaN,Que des cendres,NaN,2025-11-30,15,Sunday,31
2,1764516456,2025-11-30 15:27:00,Mathilde,f7b37e03-d216-4144-901a-cebca6426416,La nuit · Le jour,NaN,Libre !,NaN,2025-11-30,15,Sunday,27
3,1764516286,2025-11-30 15:24:00,Théadule,NaN,Un Monde Parfait,NaN,Un Monde Parfait,NaN,2025-11-30,15,Sunday,24
4,1764516117,2025-11-30 15:21:00,Louisette & Amaury,NaN,Mieux,NaN,Mieux,NaN,2025-11-30,15,Sunday,21


## Visualisations

In [29]:
time_min = max(df_kronyx["uts"].min(), df_gotwiller["uts"].min())
df_kronyx["user"] = "justin"
df_gotwiller["user"] = "gautier"

df = pd.concat([df_kronyx, df_gotwiller], ignore_index=True)
df= df[df["uts"] > time_min]
# --- Convertir la colonne de temps ---
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# --- Colonnes utiles ---
df["date"] = df["utc_time"].dt.date
df["year"] = df["utc_time"].dt.year
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()
df["week"] = df["utc_time"].dt.isocalendar().week


In [30]:
time_min
#1739983023

1739983023

In [31]:

# --- Sélecteur d'année ---
annees_disponibles = sorted(df["year"].unique())
year_selected = st.selectbox("Année à analyser", annees_disponibles)

# Filtrer l'année sélectionnée
df_year = df[df["year"] == year_selected]

jours_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 1. Regrouper par semaine / jour / utilisateur
comp = (
    df_year.groupby(["week", "weekday", "user"])
           .size()
           .reset_index(name="plays")
)

# 2. Pivot table → weekday × week × user
pivot = comp.pivot_table(
    index=["weekday", "week"],
    columns="user",
    values="plays",
    fill_value=0
).reset_index()

pivot.head()

# 3. Calcul de qui a le plus écouté
pivot["compare"] = pivot["justin"] - pivot["gautier"]

2025-12-01 21:10:32.858 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:32.859 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:32.860 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:32.868 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:32.869 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-12-01 21:10:32.879 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [32]:
pivot.head()

user,weekday,week,gautier,justin,compare
0,Friday,8,67.0,76.0,9.0
1,Friday,9,93.0,39.0,-54.0
2,Friday,10,4.0,49.0,45.0
3,Friday,11,8.0,7.0,-1.0
4,Friday,12,30.0,14.0,-16.0


In [33]:
# -1 : Gautier gagne / 0 : égalité / +1 : Justin gagne
pivot["compare_flag"] = pivot["compare"].apply(lambda x:
    1 if x > 0 else (-1 if x < 0 else 0)
)

# 4. Matrice pour la heatmap
matrix_compare = pivot.pivot(
    index="weekday",
    columns="week",
    values="compare_flag"
).reindex(jours_order)

# 5. Remap des valeurs en [0,1] pour Plotly
# -1 → 0 (jaune)
#  0 → 0.5 (gris)
# +1 → 1 (bleu)
matrix_plot = (matrix_compare + 1) / 2

# --- Labels axes ---
semaines = [f"W{w}" for w in matrix_compare.columns]
jours = matrix_compare.index.tolist()


fig = px.imshow(
    matrix_plot,
    x=semaines,
    y=jours,
    color_continuous_scale=["yellow", "lightgray", "blue"],
    aspect="auto",
)

fig.update_traces(
    hovertemplate="%{y}, %{x}<extra></extra>"
)

fig.update_layout(
    title=f"Qui écoute le plus ? — Année {year_selected}",
    xaxis_title="Semaine",
    yaxis_title="Jour de la semaine",
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()
st.plotly_chart(fig)


2025-12-01 21:10:33.741 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:33.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:33.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:33.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-12-01 21:10:33.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()